In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
#load dataframe from pickle file
clusterData = pd.read_pickle('clusteredDataFrame.pkl')
clusterData

,StartTime,Duration,Protocol,Flags,Tos,Packet,Bytes,Flows,Label,SourceIP,PORTSOURCE,DestinationIP,PORTDESTINATION,cluster,ProtocolCluster,FinalCode
2213241,2011-08-18 12:33:10.235,0.000,ICMP,ECR,0,1,1066,1,Botnet\n,147.32.96.69,NaN,147.32.84.205,NaN,0,0,0.0
2236611,2011-08-18 12:33:53.337,0.000,ICMP,ECR,0,1,1066,1,Botnet\n,147.32.96.69,NaN,147.32.84.165,NaN,0,0,0.0
2236608,2011-08-18 12:33:53.334,0.000,ICMP,UNK,0,1,1066,1,Botnet\n,147.32.84.208,NaN,147.32.96.69,NaN,0,0,0.0
2236603,2011-08-18 12:33:53.332,0.000,ICMP,ECO,0,1,1066,1,Botnet\n,147.32.84.204,NaN,147.32.96.69,NaN,0,0,0.0
2236599,2011-08-18 12:33:53.331,0.000,ICMP,ECO,0,1,1066,1,Botnet\n,147.32.84.204,NaN,147.32.96.69,NaN,0,0,0.0
2236597,2011-08-18 12:33:55.564,0.000,ICMP,UNK,0,1,1066,1,Botnet\n,147.32.84.207,NaN,147.32.96.69,NaN,0,0,0.0
2236596,2011-08-18 12:33:53.327,0.000,ICMP,UNK,0,1,1066,1,Botnet\n,147.32.84.192,NaN,147.32.96.69,NaN,0,0,0.0
2236594,2011-08-18 12:33:53.323,0.000,ICMP,UNK,0,1,1066,1,Botnet\n,147.32.84.204,NaN,147.32.96.69,NaN,0,0,0.0
2236593,2011-08-18 12:33:53.317,0.000,ICMP,UNK,0,1,1066,1,Botnet\n,147.32.84.205,NaN,147.32.96.69,NaN,0,0,0.0
2236590,2011-08-18 12:33:53.310,0.000,ICMP,ECR,0,1,1066,1,Botnet\n,147.32.96.69,NaN,147.32.84.208,NaN,0,0,0.0


In [3]:
clusterData['StartTime'] = pd.to_datetime(clusterData['StartTime'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

In [4]:
#receives as input the dataframe of a specific host and a timewindow.
#returns all states of the host during that time
def CreateStateList(hostDataset, width=20):
    time = hostDataset['StartTime']
    timeDifferenceList = []
    for i in range(len(hostDataset)):
        if i == 0:
            diff = 0
        else:
            diff = time.iloc[i] - time.iloc[i - 1]
            diff = np.ceil(diff.value / 1e6)
        timeDifferenceList.append(diff)
    hostDataset['time'] = timeDifferenceList

    
    stateList = []
    for i in range(len(hostDataset)):
        j = i
        stateList.append([])
        tempStateList = [hostDataset['FinalCode'].iloc[j]]
        duration = 0
        while True:
            try:
                duration += timeDifferenceList[j + 1]
            except:
                break
            j += 1
            if duration <= width:
                tempStateList.append(hostDataset['FinalCode'].iloc[j])
            else:
                break
        if len(tempStateList) >= 3:
            stateList[i] = tempStateList
   
    hostDataset['window_states'] = stateList
    return hostDataset

In [5]:
#given a sequence of states, computes all possible ngrams
def FindNGrams(states, n):
    res = []
    for i in range(len(states)):
        numberOfNgrams= len(states[i]) - n + 1
        for j in range(numberOfNgrams):
            res.append(states[i][j:j + n])
    return res

In [6]:
def NGramsSort(ngramList):
    nGramDict = {}
    for gram in ngramList:
        grams = str(gram)[1:-1]
        if grams in nGramDict:
            nGramDict[grams] += 1
        else:
            nGramDict[grams] = 1
    sortedNgrams = sorted(nGramDict.items(), key=lambda x: x[1], reverse=True)
    normalizedsortedNgrams = [(list[0], 1.0 * list[1] / len(ngramList)) for list in sortedNgrams ]
    return normalizedsortedNgrams

In [7]:
#calculates the distance between x and y
def calculateDistance(x, y):
    x = np.array(x)
    y = np.array(y)
    distance = sum((np.divide((x - y), (x + y) / 2)) ** 2)
    return distance

In [8]:
#computes how much do 2 different profiles match
def MatchFingerprint(finger1, finger2, NgramsConsidered):
    finger1 = finger1[0:NgramsConsidered]
    frequency1 = [pair[1] for pair in finger1]
    finger2 = {pair[0]: pair[1] for pair in finger2}
    frequency2 = []
    for i in range(NgramsConsidered):
        key = finger1[i][0]
        if key in finger2:
            frequency2.append(finger2[key])
        else:
            frequency2.append(0)
    dist = calculateDistance(frequency1, frequency2)
    return dist

In [9]:
#model one infected host
infectedHostTraining = clusterData[(clusterData['SourceIP']=='147.32.84.208') | (clusterData['DestinationIP']=='147.32.84.208')]
infectedHostTraining_States=CreateStateList(infectedHostTraining, width=90)
infectedHostTraining_States=[s for s in infectedHostTraining_States['window_states'] if len(s)>0]
infectedHostTraining_Ngrams= FindNGrams(infectedHostTraining_States,2)
infectedHostTraining=NGramsSort(infectedHostTraining_Ngrams)


C:\Users\pc1\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\pc1\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [10]:
#model one normal host
normalHostTraining = clusterData[(clusterData['SourceIP']=='147.32.84.170') | (clusterData['DestinationIP']=='147.32.84.170')]
normalHostTraining_States=CreateStateList(normalHostTraining, width=90)
normalHostTraining_States=[s for s in normalHostTraining_States['window_states'] if len(s)>0]
normalHostTraining_Ngrams= FindNGrams(normalHostTraining_States,2)
normalHostTraining=NGramsSort(normalHostTraining_Ngrams)

C:\Users\pc1\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\pc1\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
testHosts=["147.32.84.165","147.32.84.191","147.32.84.192","147.32.84.193","147.32.84.204","147.32.84.205","147.32.84.206"
          ,"147.32.84.207","147.32.84.209","147.32.84.134","147.32.84.164","147.32.87.36","147.32.80.9","147.32.87.11"]
testLabels=[1,1,1,1,1,1,1,1,1,0,0,0,0,0]

In [12]:
profMatch= np.zeros((len(testHosts),2))
#test the match of every host of the test data with one of the training hosts
for index, host in enumerate(testHosts):
    testHost = clusterData[(clusterData['SourceIP'] == host) | (clusterData['DestinationIP'] == host)]
    testStates = CreateStateList(testHost,width=90)
    testStates = [l for l in testStates['window_states'] if len(l)>0]
    testNgrams = FindNGrams(testStates, 2)
    fingerprint = NGramsSort(testNgrams)
    profMatch[index][0] = MatchFingerprint(infectedHostTraining,fingerprint,10)    
    profMatch[index][1] = MatchFingerprint(normalHostTraining,fingerprint,10)  
profMatch

C:\Users\pc1\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\pc1\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


array([[ 4.67848664, 32.67496673],
       [ 2.17929066, 31.36607183],
       [ 1.73683242, 31.07629631],
       [ 5.47682303, 29.93376138],
       [ 1.57069937, 30.9753007 ],
       [ 1.3727032 , 30.70342958],
       [ 3.9428799 , 29.915698  ],
       [ 6.14273793, 31.03994435],
       [ 2.89107668, 30.62159506],
       [39.14232578, 18.78515025],
       [26.64537752, 13.07712661],
       [37.69554387, 29.79733919],
       [39.94776727, 32.94695563],
       [40.        , 40.        ]])

In [13]:
#prdeict labels for test data hosts
predictedLabels=np.zeros(len(testLabels))
for i in range(len(predictedLabels)):
        if profMatch[i][0] <= profMatch[i][1]:
            predictedLabels[i] = 1
        else:
            predictedLabels[i] = 0
predictedLabels

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1.])

In [14]:
#evaluation
tp = 0
tn = 0
fp = 0
fn = 0
for i in range(len(testLabels)):
    if predictedLabels[i] == testLabels[i] and testLabels[i] == 1:
        tp += 1
    elif predictedLabels[i] != testLabels[i] and testLabels[i] == 1:
        fn += 1
        
    elif predictedLabels[i] == testLabels[i] and testLabels[i] == 0:
        tn += 1
    else:
        fp += 1
       
precision= float(tp)/(tp+fp)
recall=float(tp)/(tp+fn)
print("precision: "+str(precision))
print("recall: "+str(recall))
tp,fp,tn,fn

precision: 0.9
recall: 1.0


(9, 1, 4, 0)